In [15]:
import time 
import functools
import numpy as np

from pynq import (Overlay,
                  allocate)

# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

%matplotlib inline
import matplotlib.pyplot as plt

In [16]:
class GUI():
    def __init__(self):
        self.N_SNR = 7
        self.N_CH = 200
        # in seconds
        self.wait_time = 0.05
        self.overlay = []
        self.outputs = [[],[],[]]
        self.ip_path_ls = "./bitstreams/ls_ip.bit"
        self.ip_path_mmse = "./bitstreams/mmsed_ip.bit"
        # self.ip_path_dnn = "./bitstreams/dnn_ip.bit"
        self.ip_path_dnn = "./../dnn_asrar/design_1.bit"
        
    def _delete_all(self):
        self.overlay = []
        self.outputs = [[],[],[]]

    @property
    def ip_outputs(self):
        return self.outputs
    
    @ip_outputs.setter
    def ip_outputs(self, arg_outputs):
        self.outputs = arg_outputs
        
    def _load_rh_file(self, filename):
        data = np.zeros(shape=(52,52))
        flatten_data = np.zeros(shape=(52*52,))
        with open(filename, 'r') as reader:
            file_data = reader.readlines()
            for idx, line in enumerate(file_data):
                line = line.replace("{","")
                line = line.replace("}","")
                line = line[:-2]
                line = line.split(",")
                if len(line) == 1:
                    continue
                arr = np.asarray(line, dtype=np.float32)
                data[idx] = arr

        for i in range(52):
            for j in range(52):
                flatten_data[i*52+j] = data[i,j]
        return flatten_data
        
    def _load_xin_file(self, filename):
        data = np.zeros(shape=(104,200))
        with open(filename, 'r') as reader:
            file_data = reader.readlines()
            for idx, line in enumerate(file_data):
                line = line[:-1]
                line = line.split(" ")
                arr = np.asarray(line, dtype=np.float32)
                data[idx] = arr
        data = data.transpose()
        data = data[0]
        return data

    def _load_yin_file(self, filename):
        data = np.zeros(shape=(200,104))
        with open(filename, 'r') as reader:
            file_data = reader.readlines()
            for idx, line in enumerate(file_data):
                line = line.replace("{","")
                line = line.replace("}","")
                line = line[:-2]
                line = line.split(",")
                if len(line) == 1:
                    continue
                arr = np.asarray(line, dtype=np.float32)
                data[idx] = arr
        return data

    def _load_actual_file(self, filename):
        data = np.zeros(shape=(104,200))
        with open(filename, 'r') as reader:
            file_data = reader.readlines()
            for idx, line in enumerate(file_data):
                line = line[:-1]
                line = line.split(" ")
                arr = np.asarray(line, dtype=np.float32)
                data[idx] = arr

        data = data.transpose()
        return data
    
    def _load_err_file(self, filename):
        data = np.zeros(shape=(7,))
        with open(filename, 'r') as reader:
            file_data = reader.readlines()
            for idx, line in enumerate(file_data):
                line = line[:-1]
                line = line.split(" ")
                arr = np.asarray(line, dtype=np.float32)
                data[idx] = arr
        return data
    
    def _load_dnn_file(self, filename):
        data = np.zeros(shape=(104,))
        with open(filename, 'r') as reader:
            file_data = reader.readlines()
            for idx, line in enumerate(file_data):
                line = line[:-1]
                line = line.split(",")
                arr = np.asarray(line, dtype=np.float32)
                data = arr
        return data
    
    def _load_ls_ip(self):
        _overlay = Overlay(self.ip_path_ls)
        dma = _overlay.axi_dma_0
        leastsq = _overlay.leastSquare_0
        self.overlay = ["ls_ip", _overlay, [dma, leastsq]]    

    def _load_mmse_ip(self):
        _overlay = Overlay(self.ip_path_mmse)
        dma = _overlay.axi_dma_0
        mmse = _overlay.MMSED_0
        self.overlay = ["mmse_ip", _overlay, [dma, mmse]]
        
    def _load_dnn_ip(self):
        _overlay = Overlay(self.ip_path_dnn)
        dma = _overlay.axi_dma_0
        neuralnet = _overlay.neuralNetworkHwR_0
        self.overlay = ["dnn_ip", _overlay, [dma, neuralnet]]

    def _calculate_ls_ip_output(self):
        in_buffer = allocate(shape=(2*104,), dtype=np.float32)
        out_buffer = allocate(shape=(104,), dtype=np.float32)

        IP_output = np.zeros([self.N_SNR, self.N_CH, 104])
        Err_LS_hw = np.zeros([self.N_SNR,])
        Phf_hw = np.zeros([self.N_SNR,])
        xin = self._load_xin_file("./../data/XinC_1.txt")

        for n_snr in range(self.N_SNR):
            if n_snr == 0:
                print(f"\nSignal: {n_snr+1}")
            else:
                print(f"\n\nSignal: {n_snr+1}")

            yin = self._load_yin_file(f"./data/Yinc_m{n_snr+1}m.dat")
            act = self._load_actual_file(f"./data/actual{n_snr+1}.txt")

            for n_ch in range(self.N_CH):
                print(f"  Channel: {n_ch+1}\r", end="")
                self._run_ls_ip(yin[n_ch], xin, in_buffer, out_buffer)
                IP_output[n_snr,n_ch,:] = out_buffer

            # 2-norm (largest sing. value)
            Err_LS_hw[n_snr] += np.linalg.norm(act - IP_output[n_snr], 2)**2
            Phf_hw[n_snr] +=  np.linalg.norm(act, 2)**2

        Err_LS_hw /=  200
        Phf_hw /= 200
        Err_LS_normalized_hw = Err_LS_hw / Phf_hw

        self.outputs[0] = ["ls_ip", IP_output, Err_LS_normalized_hw]

    def _run_ls_ip(self, yin, xin, in_buffer, out_buffer):
        """
            self.overlay = ["ls_ip", _overlay, [dma, leastsq]]
        """
        for idx in range(2*104):
            if idx<104:
                in_buffer[idx] = yin[idx]
            else:
                in_buffer[idx] = xin[idx-104]

        self.overlay[2][1].write(0x00, 0x1)
        self.overlay[2][0].sendchannel.transfer(in_buffer)
        self.overlay[2][0].recvchannel.transfer(out_buffer)
        
    def _calculate_mmse_ip_output(self):
        in_buffer = allocate(shape=(2*104 + 52*52*2 + 1,), dtype=np.float64)
        out_buffer = allocate(shape=(104,), dtype=np.float64)

        IP_output = np.zeros([self.N_SNR, self.N_CH, 104])
        Err_MMSE_hw = np.zeros([self.N_SNR,])
        Phf_hw = np.zeros([self.N_SNR,])
        xin = self._load_xin_file("./data/XinC_1.txt")
        rhReal = self._load_rh_file("./data/realRh.dat")
        rhImag = self._load_rh_file("./data/imagRh.dat")
        snr_val = [1.0000000, 3.1622777, 10.0000000, 31.6227766, 100.0000000, 316.2277660, 1000.0000000]
        
        for n_snr in range(self.N_SNR):
            if n_snr == 0:
                print(f"\nSignal: {n_snr+1}")
            else:
                print(f"\n\nSignal: {n_snr+1}")

            yin = self._load_yin_file(f"./data/Yinc_m{n_snr+1}m.dat")
            act = self._load_actual_file(f"./data/actual{n_snr+1}.txt")

            for n_ch in range(self.N_CH):
                print(f"  Channel: {n_ch+1}\r", end="")
                self._run_mmse_ip(yin[n_ch], xin, rhReal, rhImag, snr_val[n_snr], in_buffer, out_buffer)
                IP_output[n_snr,n_ch,:] = out_buffer

            # 2-norm (largest sing. value)
            Err_MMSE_hw[n_snr] += np.linalg.norm(act - IP_output[n_snr], 2)**2
            Phf_hw[n_snr] +=  np.linalg.norm(act, 2)**2

        Err_MMSE_hw /=  200
        Phf_hw /= 200
        Err_MMSE_normalized_hw = Err_MMSE_hw / Phf_hw

        self.outputs[1] = ["mmse_ip", IP_output, Err_MMSE_normalized_hw]

    def _run_mmse_ip(self, yin, xin, rhReal, rhImag, snr_val, in_buffer, out_buffer):
        """
            self.overlay = ["mmse_ip", _overlay, [dma, mmse]]
        """
        for idx in range(2*104 + 52*52*2 + 1):
            if idx < 104:
                in_buffer[idx] = yin[idx]

            elif idx < 2*104:
                in_buffer[idx] = xin[idx-104]

            elif idx < 2*104 + 52*52:
                in_buffer[idx] = rhReal[idx-2*104]

            elif idx < 2*104 + 52*52*2:
                in_buffer[idx] = rhImag[idx-(2*104+52*52)]

            else:
                in_buffer[idx] = snr_val
        
        self.overlay[2][1].write(0x00, 0x1)
        self.overlay[2][0].sendchannel.transfer(in_buffer)
        self._wait()
        self.overlay[2][0].recvchannel.transfer(out_buffer)
        
    def _wait(self):
        start_time = time.time()
        while (time.time() - start_time) < self.wait_time:
            pass
        return True
        
    def _calculate_dnn_ip_output(self):
        in_buffer = allocate(shape=(2*104,), dtype=np.float32)
        out_buffer = allocate(shape=(104,), dtype=np.float32)

        IP_output = np.zeros([self.N_SNR, self.N_CH, 104])
        Err_DNN_hw = np.zeros([self.N_SNR,])
        Phf_hw = np.zeros([self.N_SNR,])
        xin = self._load_xin_file("./../data/XinC_1.txt")

        for n_snr in range(self.N_SNR):
            if n_snr == 0:
                print(f"\nSignal: {n_snr+1}")
            else:
                print(f"\n\nSignal: {n_snr+1}")

            xin = self._load_xin_file("./data/XinC_1.txt")
            yin = self._load_yin_file(f"./data/Yinc_m{n_snr+1}m.dat")

            for n_ch in range(self.N_CH):
                print(f"  Channel: {n_ch+1}\r", end="")
                self._run_dnn_ip(yin[n_ch], xin, in_buffer, out_buffer)        
                IP_output[n_snr,n_ch,:] = out_buffer

        denorm_IP_output = self._denomarlize(IP_output)
        Err_DNN_normalized_hw = self._caculate_error(denorm_IP_output)

        self.outputs[2] = ["dnn_ip", denorm_IP_output, Err_DNN_normalized_hw]
        
    def _denomarlize(self, IP_output):
        dnn_mean = self._load_dnn_file("./data/dnn_mean_o7.dat")
        dnn_std = self._load_dnn_file("./data/dnn_std_o7.dat")

        denorm_IP_output = np.zeros([self.N_SNR, self.N_CH, 104])

        for n_snr in range(self.N_SNR):
            for n_ch in range(self.N_CH):
                    denorm_IP_output[n_snr, n_ch, :] = np.multiply(IP_output[n_snr, n_ch, :], dnn_std) + dnn_mean

        return denorm_IP_output

    def _caculate_error(self, IP_output):
        Err_DNN_hw = np.zeros([self.N_SNR,])
        Phf_hw = np.zeros([self.N_SNR,])

        for n_snr in range(self.N_SNR):
            act = self._load_actual_file(f"./data/actual{n_snr+1}.txt")

            # 2-norm (largest sing. value)
            Err_DNN_hw[n_snr] += np.linalg.norm(act - IP_output[n_snr], 2)**2
            Phf_hw[n_snr] +=  np.linalg.norm(act, 2)**2

        Err_DNN_hw /=  200
        Phf_hw /= 200
        Err_DNN_normalized_hw = Err_DNN_hw / Phf_hw
    
        return Err_DNN_normalized_hw
    
    def _run_dnn_ip(self, yin, xin, in_buffer, out_buffer):
        """
            self.overlay = ["dnn_ip", _overlay, [dma, neuralnet]]
        """
        for idx in range(2*104):
            if idx<104:
                in_buffer[idx] = yin[idx]
            else:
                in_buffer[idx] = xin[idx-104]

        self.overlay[2][1].write(0x00, 0x1)
        self.overlay[2][0].sendchannel.transfer(in_buffer)
        self.overlay[2][0].recvchannel.transfer(out_buffer)

    def _plot_err_curves(self):
        """
            self.outputs= [out_ls_ip, out_mmse_ip, out_dnn_ip]
                out_ip = ["ip_name", IP_output, err_normalized]
        """
        flag_plt_empty = True
        for idx, out in enumerate(self.outputs):
            if len(out) != 0:
                flag_plt_empty = False
                if out[0] != "dnn_ip":
                    err_sim = self._load_err_file(f"./data/err_{out[0]}_sim.txt")
                    err_th = self._load_err_file(f"./data/err_{out[0]}_th.txt")
                    plt.semilogy(np.arange(0, self.N_SNR*5, 5), err_sim, label=f"err_{out[0]}_sim", marker="1", markersize=20)
                    plt.semilogy(np.arange(0, self.N_SNR*5, 5), err_th, label=f"err_{out[0]}_th")
                plt.semilogy(np.arange(0, self.N_SNR*5, 5), out[2], label=out[0])
        
        if not flag_plt_empty:
            plt.legend(loc="upper right")

        plt.xlabel("Preamble SNR")
        plt.ylabel("Average Error per subcarrier")
        plt.show()
    
    def _create_widgets(self):
        self.box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

        self.ip_selector = widgets.Dropdown(
                        options=['Least Square IP', 'MMSE IP', 'DNN IP'],
                        value='Least Square IP',
                        description='Load IP:')

        self.btn_load_IP = widgets.Button(description='Load the IP')
        self.btn_run_selected_ip = widgets.Button(description='Run IP')
        self.btn_plot = widgets.Button(description='Plot')
        self.btn_clear_outputs = widgets.Button(description='Reset')
        self.out = widgets.Output()
        
        self.btn_load_IP.on_click(self._btn_load_IP_on_click)
        self.btn_run_selected_ip.on_click(self._btn_run_ip_on_click)
        self.btn_plot.on_click(self._btn_plot_on_click)
        self.btn_clear_outputs.on_click(self._btn_clear_outputs_on_click)
        
    def _btn_load_IP_on_click(self, change):
         # "linking function with output"
        with self.out:
            # what happens when we press the button
            clear_output()
            if self.ip_selector.value == "Least Square IP":
                self._load_ls_ip()
                print("Least Square IP loaded!")

            elif self.ip_selector.value == "MMSE IP":
                self._load_mmse_ip()
                print("MMSE IP loaded!")

            elif self.ip_selector.value == "DNN IP":
                self._load_dnn_ip()
                print("DNN IP loaded!")  

    def _btn_run_ip_on_click(self, change):
        # "linking function with output"
        with self.out:
        # what happens when we press the button
            clear_output()
            if self.overlay[0] == "ls_ip":
                print("Running Least Square IP...") 
                self._calculate_ls_ip_output()
            elif self.overlay[0] == "mmse_ip":
                print("Running MMSE IP...") 
                self._calculate_mmse_ip_output()    
            elif self.overlay[0] == "dnn_ip":
                print("Running DNN IP...") 
                self._calculate_dnn_ip_output()
            print("\n\nIP run finished!")

    def _btn_plot_on_click(self, change):
        # "linking function with output"
        with self.out:
        # what happens when we press the button
            clear_output()
            self._plot_err_curves()
            
    def _btn_clear_outputs_on_click(self, change):
        # "linking function with output"
        with self.out:
        # what happens when we press the button
            clear_output()
            self._delete_all()
            print("All outputs cleared!")
    
    def display_widgets(self):
        self._create_widgets()
        display(
            widgets.HBox(
                [
                    self.ip_selector, 
                    self.btn_load_IP, 
                    self.btn_run_selected_ip, 
                    self.btn_plot,
                    self.btn_clear_outputs,
                    self.out
                ], 
                layout=self.box_layout)
        )

In [17]:
x = GUI()
x.display_widgets()

In [ ]:
x.outputs